In [ ]:
import param
import numpy as np
import pandas as pd
import holoviews as hv
import networkx as nx
from bokeh.models import HoverTool

hv.notebook_extension('bokeh', 'matplotlib')

%opts Graph [width=400 height=400]

Visualizing and working with network graphs is a common problem in many different disciplines. HoloViews provides the ability to represent and visualize graphs very simply and easily with facilities for interactively exploring the nodes and edges of the graph, especially using the bokeh plotting interface.

The ``Graph`` ``Element`` differs from other elements in HoloViews in that it consists of multiple sub-elements. The data of the ``Graph`` element itself are the abstract edges between the nodes, on its own this abstract graph cannot be visualized. In order to visualize it we need to give each node in the ``Graph`` a concrete ``x`` and ``y`` position in form of the ``Nodes``. The abstract edges and concrete node positions are sufficient to render the ``Graph`` by drawing straight-line edges between the nodes. In order to supply explicit edge paths we can also declare ``NodePaths``, providing explicit coordinates for each edge to follow.

To summarize a ``Graph`` consists of three different components:

* The ``Graph`` itself holds the abstract edges stored as a table of node indices.
* The ``Nodes`` hold the concrete ``x`` and ``y`` positions of each node along with a node ``index``.
* The ``NodePaths`` can optionally be supplied to declare explicit node paths.

#### A simple Graph

Let's start by declaring a very simple graph laid out in a circle:

In [ ]:
N = 8
node_indices = np.arange(N)

# Declare abstract edges
start = np.zeros(N)
end = node_indices

### start of layout code
circ = np.pi/N*node_indices*2
x = np.cos(circ)
y = np.sin(circ)

simple_graph = hv.Graph(((start, end), (x, y, node_indices))).redim.range(x=(-1.2, 1.2), y=(-1.2, 1.2))
simple_graph

#### Accessing the nodes and edges

We can easily access the ``Nodes`` and ``NodePaths`` on the ``Graph`` element using the corresponding properties:

In [ ]:
simple_graph.nodes + simple_graph.nodepaths

#### Supplying explicit paths

Next we will extend this example by supplying explicit edges:

In [ ]:
def bezier(start, end, control, steps=np.linspace(0, 1, 100)):
    return (1-steps)*(1-steps)*start + 2*(1-steps)*steps*control+steps*steps*end

paths = []
for node_index in node_indices:
    ex, ey = x[node_index], y[node_index]
    paths.append(np.column_stack([bezier(x[0], ex, 0), bezier(y[0], ey, 0)]))
    
bezier_graph = hv.Graph(((start, end), (x, y, node_indices), paths)).redim.range(x=(-1.2, 1.2), y=(-1.2, 1.2))
bezier_graph

## Interactive features

#### Hover and selection policies

Thanks to Bokeh we can reveal more about the graph by hovering over the nodes and edges. The ``Graph`` element provides an ``inspection_policy`` and a ``selection_policy``, which define whether hovering and selection highlight edges associated with the selected node or nodes associated with the selected edge, these policies can be toggled by setting the policy to ``'nodes'`` and ``'edges'``.

In [ ]:
%%opts Graph [tools=['hover']]
bezier_graph.opts(plot=dict(inspection_policy='edges'))

In addition to changing the policy we can also change the colors used when hovering and selecting nodes:

In [ ]:
%%opts Graph [tools=['hover', 'box_select']] (edge_hover_line_color='green' node_hover_fill_color='red')
bezier_graph.opts(plot=dict(inspection_policy='nodes'))

#### Additional information

We can also associate additional information with the nodes of a graph. By constructing the ``Nodes`` explicitly we can declare an additional value dimension, which we can reveal when hovering and color the nodes by:

In [ ]:
%%opts Graph [tools=['hover'] color_index='Type'] (cmap='Set1')
nodes = hv.Nodes((x, y, node_indices, ['Input']+['Output']*(N-1)), vdims=['Type'])
hv.Graph(((start, end), nodes, paths)).redim.range(x=(-1.2, 1.2), y=(-1.2, 1.2))

## Working with NetworkX

NetworkX is a very useful library when working with network graphs and the Graph Element provides ways of importing a NetworkX Graph directly. Here we will load the Karate Club graph and use the ``circular_layout`` function provided by NetworkX to lay it out:

In [ ]:
%%opts Graph [tools=['hover']]
import networkx as nx
G = nx.karate_club_graph()
hv.Graph.from_networkx(G, nx.layout.circular_layout).redim.range(x=(-1.2, 1.2), y=(-1.2, 1.2))

## Real world graphs

As a final example let's look at a slightly larger graph. We will load a dataset of a Facebook network consisting a number of friendship groups identified by their ``'circle'``. We will load the edge and node data using pandas and then color each node by their friendship group using many of the things we learned above.

In [ ]:
%%opts Graph [width=800 height=800 xaxis=None yaxis=None tools=['hover'] color_index='circle']
%%opts Graph (node_size=10 edge_line_width=1)
colors = ['#000000']+hv.Cycle('Category20').values
edges_df = pd.read_csv('../examples/assets/fb_edges.csv')
fb_nodes = hv.Nodes(pd.read_csv('../examples/assets/fb_nodes.csv')).sort()
fb_graph = hv.Graph((edges_df, fb_nodes), label='Facebook Circles')
fb_graph.redim.range(x=(-0.05, 1.05), y=(-0.05, 1.05)).opts(style=dict(cmap=colors))